# Usare le risorse di calcolo

Quando si esegue uno script come processo di Azure Machine Learning, è necessario definire il contesto di esecuzione per l'esecuzione del processo. Una configurazione chiave è la destinazione di calcolo in cui verrà eseguito lo script. Potrebbe trattarsi della workstation locale (in questo caso dell'istanza di calcolo) o di una destinazione di calcolo remota, ad esempio il cluster di calcolo gestito di Azure Machine Learning di cui è stato effettuato il provisioning su richiesta.

In questo notebook si creerà un cluster di calcolo ed si esploreranno le destinazioni di calcolo per i processi.

## Prima di iniziare

Per eseguire il codice in questo notebook, è necessaria la versione più recente del pacchetto  **azureml-ai-ml** . Eseguire la cella seguente per verificare che sia installata.

> **Nota**:
> Se il pacchetto **azure-ai-ml** non è installato, eseguire `pip install azure-ai-ml` per installarlo.

In [ ]:
## Connettersi all'area di lavoro

Dopo aver installato i pacchetti SDK necessari, è ora possibile connettersi all'area di lavoro.

Per connettersi a un'area di lavoro, sono necessari parametri di identificatore: ID sottoscrizione, nome del gruppo di risorse e nome dell'area di lavoro. Il nome del gruppo di risorse e il nome dell'area di lavoro sono già compilati automaticamente. Per completare il comando è necessario solo l'ID sottoscrizione.

Per trovare i parametri necessari, fare clic sulla sottoscrizione e sul nome dell'area di lavoro in alto a destra di Studio. Verrà aperto un riquadro a destra.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiare l'ID sottoscrizione e sostituire **YOUR-SUBSCRIPTION-ID** con il valore copiato. </p>

## Creare un cluster di elaborazione

In molti casi, le risorse di calcolo locali potrebbero non essere sufficienti per elaborare un esperimento complesso o a esecuzione prolungata che deve elaborare un volume elevato di dati; e può essere utile sfruttare la possibilità di creare e usare in modo dinamico le risorse di calcolo nel cloud. Azure Machine Learning supporta una gamma di destinazioni di calcolo, che è possibile definire nel flusso di lavoro e usare per eseguire i processi; pagamento per le risorse solo quando vengono usate.

È possibile creare un cluster di calcolo in [studio di Azure Machine Learning](https://ml.azure.com) usando Python SDK o l'interfaccia della riga di comando di Azure. La cella di codice seguente controlla l'esistenza di nomi `aml-cluster`di cluster di calcolo e, se non esiste, crea l'area di lavoro.

In [ ]:
Dopo aver creato un cluster di calcolo, è possibile modificare solo la configurazione per:

- `min_instances`: numero minimo di nodi
- `max_instances`: numero massimo di nodi
- `idle_time_before_scale_down`: tempo di inattività prima della riduzione delle prestazioni

Attualmente, il cluster `aml-cluster` di calcolo può eseguire solo una scalabilità massima di un nodo. Si modificherà ora a due per consentire il calcolo parallelo.

In [ ]:
Quando il cluster di calcolo viene aggiornato, è possibile verificarne la configurazione stampandone gli attributi.

## Creare uno script per eseguire il training di un modello

Per eseguire il training di un modello, creare prima di tutto lo script **diabetes_training.py** nella cartella **src** . Lo script usa il file **diabetes.csv** nella stessa cartella dei dati di training.

In [ ]:
## Eseguire un processo in un cluster di calcolo

A questo momento, si è pronti per eseguire il processo nel cluster di calcolo creato.

> **Nota**:
> L'avvio del processo richiederà del tempo perché il cluster di calcolo dovrà essere ridimensionato da zero a un nodo. Quando il cluster di calcolo è pronto, lo script verrà eseguito. Al termine del processo, il cluster di calcolo eseguirà il ridimensionarsi fino a zero nodi. È possibile esaminare lo stato del cluster di calcolo nella pagina **Calcolo** .

After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="aml-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [ ]:
cpu_cluster = ml_client.compute.get("aml-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)